In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing import sequence
from string import punctuation as en_punctuation
import re
from collections import Counter
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import issparse
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
train_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/train.csv'
test_dir = r'/home/yw/study/Competition/Avito-Demand-Prediction-Challenge/data/test.csv'

In [12]:
train=pd.read_csv(train_dir)

In [15]:
np.sqrt(0.05425)
np.sqrt(0.05392)

0.23220680437919988

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1503424 entries, 0 to 1503423
Data columns (total 18 columns):
item_id                 1503424 non-null object
user_id                 1503424 non-null object
region                  1503424 non-null object
city                    1503424 non-null object
parent_category_name    1503424 non-null object
category_name           1503424 non-null object
param_1                 1441848 non-null object
param_2                 848882 non-null object
param_3                 640859 non-null object
title                   1503424 non-null object
description             1387148 non-null object
price                   1418062 non-null float64
item_seq_number         1503424 non-null int64
activation_date         1503424 non-null object
user_type               1503424 non-null object
image                   1390836 non-null object
image_top_1             1390836 non-null float64
deal_probability        1503424 non-null float64
dtypes: float64(3), int